In [3]:
import re
import os
import pathlib
import json

import plotnine
import requests
import pandas as pd

In [10]:
# results_dir = pathlib.Path("../results").resolve()
experiment = "01_http_max_throughput"

curr_dir = pathlib.Path().resolve()
results_dir = 


PosixPath('/Users/richardbieringa/Projects/mesh-bench/viz')

In [42]:
results_dir = pathlib.Path("../results").resolve()
print(results_dir)

experiments = {}

name_re = re.compile(".*\d{2}_[a-z_]+\/([a-z]+)_([a-z]+)_(\d+|MAX)_(\d+)_(\d+).*json")

for root, dirs, files in os.walk(results_dir, topdown=False):

    experiment = pathlib.Path(root).name

    # Only parse experiment directories
    if not re.search("\d{2}_(http|grpc)", experiment):
        continue

    print(f"Parsing experiment: {experiment} -> {len(files)}")


    experiments[experiment] = []
    for file in files:
        p = (pathlib.Path(root) / file).resolve()
        matches = name_re.match(str(p))

        with open(p) as f:
            data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["mesh"] = matches[2]
            df["payload"] = matches[4]
            df["repetition"] = matches[5]
            experiments[experiment].append(df)


/Users/richardbieringa/Projects/mesh-bench/results
Parsing experiment: 02_http_constant_throughput -> 100
Parsing experiment: 03_http_payload -> 75
Parsing experiment: 04_grpc_max_throughput -> 25
Parsing experiment: 01_http_max_throughput -> 25


In [51]:

df = pd.concat(experiments["01_http_max_throughput"])
df


,RunType,Labels,StartTime,RequestedQPS,RequestedDuration,ActualQPS,ActualDuration,NumThreads,Version,Exactly,...,HeaderSizes.Min,HeaderSizes.Max,HeaderSizes.Sum,HeaderSizes.Avg,HeaderSizes.StdDev,HeaderSizes.Data,HeaderSizes.Percentiles,mesh,payload,repetition
0,HTTP,,2022-06-12T14:31:28.384812039Z,max,10m0s,565.151631,600067276129,32,1.28.0,0,...,75,75,25434675,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,traefik,0,3
0,HTTP,,2022-06-12T13:58:20.928460602Z,max,10m0s,4904.844412,600003741777,32,1.28.0,0,...,209,210,615079995,209.002946,0.054198,"[{'Start': 209, 'End': 210, 'Percent': 100, 'C...",None,istio,0,5
0,HTTP,,2022-06-12T13:28:20.317549217Z,max,10m0s,4913.836228,600002495653,32,1.28.0,0,...,209,210,616205631,209.002715,0.052036,"[{'Start': 209, 'End': 210, 'Percent': 100, 'C...",None,istio,0,2
0,HTTP,,2022-06-12T12:44:30.55607617Z,max,10m0s,11499.706401,600002448682,32,1.28.0,0,...,75,75,517488900,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,linkerd,0,3
0,HTTP,,2022-06-12T12:24:30.158244439Z,max,10m0s,11456.248625,600003476253,32,1.28.0,0,...,75,75,515534175,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,linkerd,0,1
0,HTTP,,2022-06-12T15:50:13.324269143Z,max,10m0s,21281.654675,600002358610,32,1.28.0,0,...,75,75,957678225,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,cilium,0,5
0,HTTP,,2022-06-12T10:58:22.869262366Z,max,10m0s,25180.428357,600001985110,32,1.28.0,0,...,75,75,1133123025,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,baseline,0,1
0,HTTP,,2022-06-12T14:41:28.694553015Z,max,10m0s,570.605233,600057588093,32,1.28.0,0,...,75,75,25679700,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,traefik,0,4
0,HTTP,,2022-06-12T15:10:12.197840034Z,max,10m0s,21105.985743,600002679527,32,1.28.0,0,...,75,75,949773600,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,cilium,0,1
0,HTTP,,2022-06-12T13:18:20.107473828Z,max,10m0s,4874.675693,600014890118,32,1.28.0,0,...,209,210,611308584,209.003105,0.055637,"[{'Start': 209, 'End': 210, 'Percent': 100, 'C...",None,istio,0,1


In [52]:
# Construct aggregator functions (default to take first)
aggregator = {}
for key in df.columns: 
    aggregator[key] = "first"


# Overwrite aggregators
overwrite = {
    "ActualQPS": "mean",
    "repetition": "count",
    "DurationHistogram.Count": "mean",
    "DurationHistogram.Min": "mean",
    "DurationHistogram.Max": "mean",
    "DurationHistogram.Sum": "mean",
    "DurationHistogram.Avg": "mean",
    "DurationHistogram.StdDev": "mean",
    "DurationHistogram.Data": "sum",
    # "DurationHistogram.Percentiles": "mean",
    "ErrorsDurationHistogram.Count": "mean",
    "ErrorsDurationHistogram.Min": "mean",
    "ErrorsDurationHistogram.Max": "mean",
    "ErrorsDurationHistogram.Sum": "mean",
    "ErrorsDurationHistogram.Avg": "mean",
    "ErrorsDurationHistogram.StdDev": "mean",
    "ErrorsDurationHistogram.Data": "sum",
    # "ErrorsDurationHistogram.Percentiles": "mean"
}

for key, value in overwrite.items():
    aggregator[key] = value


df = df.groupby("mesh").aggregate(aggregator)
df

,RunType,Labels,StartTime,RequestedQPS,RequestedDuration,ActualQPS,ActualDuration,NumThreads,Version,Exactly,...,HeaderSizes.Min,HeaderSizes.Max,HeaderSizes.Sum,HeaderSizes.Avg,HeaderSizes.StdDev,HeaderSizes.Data,HeaderSizes.Percentiles,mesh,payload,repetition
mesh,,,,,,,,,,,,,,,,,,,,,
baseline,HTTP,,2022-06-12T10:58:22.869262366Z,max,10m0s,25019.497379,600001985110,32,1.28.0,0,...,75,75,1133123025,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,baseline,0,5
cilium,HTTP,,2022-06-12T15:50:13.324269143Z,max,10m0s,21223.002674,600002358610,32,1.28.0,0,...,75,75,957678225,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,cilium,0,5
istio,HTTP,,2022-06-12T13:58:20.928460602Z,max,10m0s,4897.250102,600003741777,32,1.28.0,0,...,209,210,615079995,209.002946,0.054198,"[{'Start': 209, 'End': 210, 'Percent': 100, 'C...",None,istio,0,5
linkerd,HTTP,,2022-06-12T12:44:30.55607617Z,max,10m0s,11458.877746,600002448682,32,1.28.0,0,...,75,75,517488900,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,linkerd,0,5
traefik,HTTP,,2022-06-12T14:31:28.384812039Z,max,10m0s,565.264404,600067276129,32,1.28.0,0,...,75,75,25434675,75.000000,0.000000,"[{'Start': 75, 'End': 75, 'Percent': 100, 'Cou...",None,traefik,0,5
